<a href="https://colab.research.google.com/github/BillGu19/Bass/blob/master/ChronWebScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import requests
import math
from bs4 import BeautifulSoup

In [136]:
#collects individual article urls from specified page
def collect_url(page, num_articles, url_replace, date):
  URL = 'https://www.dukechronicle.com/section/news?page=' + page + '&per_page=' + num_articles
  if(url_replace != ''):
    URL = url_replace
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')

  url_html = soup.select('article.art-left.mb-4') 

  url_list = []
  url_set = set()
  for article_html in url_html:
    url_list.append(str(article_html))
  for article in url_list:
    ind = article.index("https://www.dukechronicle.com/article/")
    url = ''
    while(True):
      url += article[ind]
      if(article[ind+1] == '"'):
        break
      ind += 1
    if(date != ''):
      end_ind = 45
      if(date[5::] == ''):
        end_ind = 43
      if(url[38:end_ind:] == DATE):
        url_set.add(url)
    else:
      url_set.add(url)
  return url_set


In [138]:
#cleans article of HTML tags and links
def clean_full_article(article):
  clean_article = ''
  not_in_par = True
  for c in article:
    if(c == '<'):
      not_in_par = False
    if(not_in_par):
      clean_article += c
    if(c == '>'):
      not_in_par = True
  ind = clean_article.rfind('facebook')
  clean_article = clean_article[:ind:]
  return clean_article

In [139]:
#extract full article from article link
#returns number of occurences for specified WORDS
def count_article(URL, words):
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')

  results = soup.find('article', class_='full-article')
  title = ''
  try:
    title = results.find('h1')
  except:
    print('no title in: ' + URL)
  headers = ''
  try:
    headers = results.find('h3')
  except:
    print('no headers in: ' + URL)
  article = ''
  try:
    article = results.find_all('p')
  except:
    print('no article text in: ' + URL)
  title_str = ''
  for x in title:
      title_str = title_str + str(x)
  headers_str = ''
  for x in headers:
    headers_str = headers_str + str(x)
  article_str = ''
  for x in article:
    article_str = article_str + str(x)
  full_article = title_str + headers_str + article_str
  clean_article = clean_full_article(full_article)
  word_dictionary = {}
  for word in words:
    word_dictionary[word] = clean_article.count(word)
  if(PRINT_ARTICLES == True):
    print(URL)
    print(str(word_dictionary) + '\n')
  return word_dictionary

In [140]:
#finds roughly the correct page range for a given year/month
#oldest date is October 1990
#year in YYYY and month in MM -> 01 for January
#month can be left empty -> ''
#returns url pointing to first instance of given date
def find_page_date(year, month):
  page = 1
  beg_page = ''
  end_page = ''
  in_date = True
  while(page < 1874):
    url_set = collect_url(str(page), num_articles='10', url_replace='', date='')
    year_month = year + '/' + month
    should_break = False
    for x in url_set:
      end_ind = 45
      if(month == ''):
        end_ind = 43
      if(in_date == True and x[38:end_ind:] == year_month):
        beg_page = str(page)
        in_date = False
        break
      elif(in_date == False and x[38:end_ind:] != year_month):
        end_page = str(page)
        should_break = True
        break
    if(should_break == True):
      break
    page += 1
  diff = find_diff(int(beg_page), int(end_page))
  beg_page = str(math.ceil(int(beg_page)/(diff)))
  num_articles = diff*10
  return 'https://www.dukechronicle.com/section/news?page=' + beg_page + '&per_page=' + str(num_articles)

In [147]:
#each date format must include year and month -> YYYY/MM
#beg_date needs to be more recent than end_date
#end_date needs to be a month older than your target
#eg. range you want -> (2020/08, 2020/06)
#then input should be (2020/08, 2020/05)
def find_page_date_range(beg_date, end_date):
  page = 1
  beg_page = ''
  end_page = ''
  in_date = True
  while(page < 1874):
    url_set = collect_url(str(page), num_articles='10', url_replace='', date='')
    should_break = False
    for x in url_set:
      end_ind = 45
      if(in_date == True and x[38:end_ind:] == beg_date):
        beg_page = str(page)
        in_date = False
        break
      elif(in_date == False and x[38:end_ind:] == end_date):
        end_page = str(page)
        should_break = True
        break
    if(should_break == True):
      break
    page += 1
  diff = find_diff(int(beg_page), int(end_page))
  beg_page = str(math.ceil(int(beg_page)/(diff)))
  num_articles = diff*10
  return 'https://www.dukechronicle.com/section/news?page=' + beg_page + '&per_page=' + str(num_articles)

In [141]:
def find_diff(beg_page, end_page):
  diff = end_page - beg_page + 1
  while(True):
    sum = 1
    do_break = True
    while(sum < end_page):
      sum += diff
      if(sum > beg_page and sum <= end_page):
        do_break = False
        break
    if(do_break == True):
      break
    else:
      diff += 1
  return diff

ALL THE GOOD STUFF HAPPENS DOWN HERE (Everything you need to worry about @Shan <3)

In [142]:
#finds roughly the correct page range for a given year/month
#oldest date is October 1990
#year in YYYY and month in MM -> 01 for January
#returns url pointing to first instance of given date
YEAR = '2020'
MONTH = '08'

find_page_date(YEAR, MONTH)

'https://www.dukechronicle.com/section/news?page=1&per_page=60'

In [148]:
#each date format must include year and month -> YYYY/MM
#beg_date needs to be more recent than end_date
#end_date needs to be a month older than your target
#eg. range you want -> (2020/08, 2020/06), then input should be (2020/08, 2020/05)

BEGIN_DATE = '2020/08'
END_DATE = '2020/05'

find_page_date_range(BEGIN_DATE, END_DATE)

'https://www.dukechronicle.com/section/news?page=1&per_page=110'

In [149]:
#variable inputs

#page number of Duke Chronicle All News
PAGE = '1' 

#number of articles to display per page
ARTICLE_PER_PAGE = '10' 

#can provide your own dukechronicle URL, leave empty if you just want to specify page and number of articles
#link should be in the format 'https://www.dukechronicle.com/section/news?page=PAGE&per_page=ARTICLE_PER_PAGE'
URL_REPLACE = 'https://www.dukechronicle.com/section/news?page=1&per_page=60'

#words to track in articles 
WORDS = ['Duke', 'University'] 

#specify articles on page with only this date, leave blank if not specifying
#format YYYY/MM
#can also leave month empty and just filter for year, eg. '2020/'
DATE = '2020/08'

#print individual articles and word count (articles are formatted with title, headers, and then text)
PRINT_ARTICLES = True

In [146]:
#calls functions written above
url_set = collect_url(PAGE, ARTICLE_PER_PAGE, URL_REPLACE, DATE)
occurence_dictionary = {}
for w in WORDS:
  occurence_dictionary[w] = 0
for article_url in url_set:
  article_occurences = count_article(article_url, WORDS)
  for x in article_occurences:
    occurence_dictionary[x] += article_occurences[x]
print('Total word counts: ' + str(occurence_dictionary))

https://www.dukechronicle.com/article/2020/08/duke-university-announce-11-postive-covid-results-tests
{'Duke': 9, 'University': 1}

https://www.dukechronicle.com/article/2020/08/duke-university-know-my-name-chanel-miller-class-2024
{'Duke': 3, 'University': 1}

https://www.dukechronicle.com/article/2020/08/duke-university-researchers-test-vaccine-treat-breast-cancer
{'Duke': 3, 'University': 0}

https://www.dukechronicle.com/article/2020/08/200827-boyd-balasubramanian-ennis
{'Duke': 4, 'University': 3}

https://www.dukechronicle.com/article/2020/08/duke-university-compact-detailed-emailed-community-tuesday-coronavirus
{'Duke': 20, 'University': 1}

https://www.dukechronicle.com/article/2020/08/faculty-students-search-for-solutions-issue-child-care-during-pandemic
{'Duke': 6, 'University': 3}

https://www.dukechronicle.com/article/2020/08/duke-university-enforcement-education-safety-regulations-pandemic
{'Duke': 36, 'University': 5}

https://www.dukechronicle.com/article/2020/08/duke-un